In [ ]:
# import packages
import numpy as np
from scipy.io import loadmat
import pandas as pd

from rpy2.robjects import r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import matplotlib.pyplot as plt
bf_package=importr('BayesFactor')


In [ ]:
# chance decoding
chance=0.5
## load data from matlab struct [such as cosmomvpa results file]
x = loadmat('../data_colour/ds_stacked_realcolour.mat',squeeze_me=True,struct_as_record=False)
data=x['ds_stacked_realcolour']
df=pd.DataFrame(data.samples)

## load data from csv file [this should be participants x timepoints accuracy matrix]
# df = pd.read_csv('../data_colour/res.csv',header=None)

# loop over timepoints, make decoding accuracy into effect size and convert to an r object
n_timepoints = df.shape[1]
df_norm = pd.DataFrame(np.empty_like(df))
for t in range(n_timepoints):
  df_norm[t]=[(i - chance) for i in df[t]]
 
with localconverter(ro.default_converter + pandas2ri.converter):
  r_data = ro.conversion.py2rpy(df_norm)

In [ ]:
# loop over timepoints
bf=[]
for t in range(n_timepoints):
    results=bf_package.ttestBF(x=r_data[t],mu=0,rscale='medium',nullInterval=[0.5,float('inf')])
    bf.append(np.asarray(r['as.vector'](results))[0])


In [21]:
# plot as a quick peak
%matplotlib qt
x = np.arange(-100,805,5)
y = bf
plt.stem(x, y,bottom=1,linefmt=None, markerfmt=None, basefmt=None)
plt.yscale('log')
ax = plt.gca()
ax.set_ylim(bottom=10**-10, top=10**10)
